In [ ]:
# По сгенерированному списку примеров запросов и ground truth ответов в csv-файле создается набор 
# сгенерированных TTS-моделью файлов.
# Поясняющий датасет при этом имеет вид:
# user_text, user_audio, assistant_text

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # This should be False if using ROCm-only PyTorch

if torch.cuda.is_available():
    print("PyTorch GPU (CUDA/HIP) is available.")
    # You can further check if it is specifically HIP using torch.version.hip
    if torch.version.hip:
        print("Backend is HIP (AMD GPU).")
    elif torch.version.cuda:
        print("Backend is CUDA (NVIDIA GPU).")
else:
    print("PyTorch GPU not available (using CPU).")

2.9.1+rocm6.4
True
PyTorch GPU (CUDA/HIP) is available.
Backend is HIP (AMD GPU).


In [2]:
import torch

# Сохраняем оригинальную функцию torch.load
original_torch_load = torch.load

# Создаем новую функцию, которая устанавливает weights_only=False
def custom_torch_load(*args, **kwargs):
    # Устанавливаем weights_only=False
    kwargs['weights_only'] = False
    return original_torch_load(*args, **kwargs)

# Заменяем torch.load на нашу функцию
torch.load = custom_torch_load

In [3]:
import os

# Перед импортом любых библиотек ML
os.environ.update({
    # Принудительно искать алгоритмы с workspace
    'MIOPEN_FIND_ENFORCE': '3',
    
    # Кэшировать найденные алгоритмы
    #'MIOPEN_USER_DB_PATH': '/tmp/miopen',
    
    # Разрешить выделение workspace
    'MIOPEN_DISABLE_WORKSPACE_FALLBACK': '0',
    
    # Увеличить лимит workspace
    'MIOPEN_WORKSPACE_LIMIT': '1073741824',  # 1 GB
    
    # Режим отладки (можно временно включить)
    'MIOPEN_DEBUG_CONV_GEMM': '0',
    
    # Уровень логирования (только ошибки)
    'MIOPEN_LOG_LEVEL': '0',
    
    # Для PyTorch (если используете)
    'PYTORCH_MIOPEN_CACHE': '1',
    'PYTORCH_ROCM_ARCH': 'gfx1100',  # Укажите вашу архитектуру GPU
})

torch.cuda.empty_cache()

# Настроить backend для сверточных операций
torch.backends.cudnn.enabled = True  # Используется и для ROCm
torch.backends.cudnn.benchmark = True  # Автоматически ищет лучшие алгоритмы
torch.backends.cudnn.deterministic = False  # Разрешить разные алгоритмы

# Очистить кэш программно
import torch
if hasattr(torch, 'miopen'):
    torch.miopen.clear()

In [4]:
# Для Coqui TTS
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Убрать логи TensorFlow
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Для NVIDIA (но может влиять на ROCm поведение)
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '0'

In [5]:
os.environ["COQUI_TOS_AGREED"] = "1"
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/home/m1/nvme/p311r64tts/lib/python3.11/site-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
load_tts_model_by_name check:
model_path = None
config_path = None
vocoder_path = None
vocoder_config_path = None
model_dir = /home/m1/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


/home/m1/nvme/p311r64tts/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/m1/nvme/p311r64tts/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


 > Using model: xtts
sytesizer.py  output_sample_rate = 24000


In [6]:
# Теперь считаем csv с текстом для генерации
from pathlib import Path
import pandas as pd
import os

data_path = "audio_assistant_queries.csv"

df = pd.read_csv(
    data_path,
    delimiter=",",
)






In [7]:
out_dir = 'audioplayer_va/'
ref_speakers_dir = './speakers/'
ref_speakers_list = [f for f in os.listdir(ref_speakers_dir) if f.endswith('.wav')]




In [ ]:
# Функция для генерации

#def generate(text:str="",speakers:list = [], )

In [10]:
from tqdm import tqdm
from collections import namedtuple
from pathlib import Path
# user_text, user_audio, assistant_text

audioplayer_va_dataset = []
lang = 'en'

num = 0
for row in tqdm(df.itertuples()):
    #if num > 15:
    #    break
    text_request = row.Request
    text_gtresponse = row.Response
    
    #Будем генерировать для разных speakers
    for spk in ref_speakers_list:
        num+=1                    
        filename = f"{num:05}.wav"
        file_path = Path(out_dir) / filename
        spk_path = Path(ref_speakers_dir) / spk 
        #print (f"filepath = {file_path}, spk = {spk_path}")
        
        tts.tts_to_file(text=text_request,
                file_path=file_path,                
                speaker_wav=spk_path,
                language=lang,
                split_sentences=True
                )
        
        audioplayer_va_dataset.append((text_request,filename,text_gtresponse))

0it [00:00, ?it/s]

 > Text splitted to sentences.
['Stop']
 > Processing time: 17.62901258468628
 > Real-time factor: 2.530463802548774
 > Text splitted to sentences.
['Stop']


1it [00:18, 18.19s/it]

 > Processing time: 0.5436239242553711
 > Real-time factor: 0.43330348213674574
 > Text splitted to sentences.
['Stop playing']
 > Processing time: 6.995113849639893
 > Real-time factor: 2.3079102882535256
 > Text splitted to sentences.
['Stop playing']


2it [00:33, 16.51s/it]

...

 > Processing time: 1.3830912113189697
 > Real-time factor: 0.49011894461274236
 > Text splitted to sentences.
["What's happening in the news today?"]
 > Processing time: 1.3909471035003662
 > Real-time factor: 0.49290279686588895
 > Text splitted to sentences.
["What's happening in the news today?"]


489it [1:12:28,  2.81s/it]

 > Processing time: 1.0800518989562988
 > Real-time factor: 0.47691333651046114
 > Text splitted to sentences.
["Summarize today's headlines"]
 > Processing time: 1.278003454208374
 > Real-time factor: 0.48693627601075906
 > Text splitted to sentences.
["Summarize today's headlines"]


490it [1:12:30,  8.88s/it]

 > Processing time: 1.05153489112854
 > Real-time factor: 0.4740420418176379


In [11]:
# Запишем csv
import csv
csv_filename = Path(out_dir) / 'audioplayer_va.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    writer.writerow(['user_text', 'user_audio', 'assistant_text'])
    for user_text, user_audio, assistant_text in audioplayer_va_dataset:
        writer.writerow([user_text, user_audio, assistant_text])
        
print("CSV file created successfully")

CSV file created successfully
